In [ ]:
import pathlib

from cfdmod.use_cases.snapshot.camera import take_snapshot
from cfdmod.use_cases.snapshot.config import SnapshotConfig
from cfdmod.use_cases.snapshot.process_image import process_image
from cfdmod.utils import create_folder_path
import warnings

project_path = pathlib.Path("/home/ubuntu/Documentos/Repositories/insight/Docker/local/volume/PrologisCajamar4_final")

for block_lbl in ["G100", "G200", "G300"]:
    cfg_path = project_path / block_lbl / "configs" / "snapshot_params.yaml"
    cfg = SnapshotConfig.from_file(cfg_path)

    output_path = project_path / block_lbl / "images"
    create_folder_path(output_path)
    
    for polydata_cfg in cfg.polydata:
        vtp_path = pathlib.Path(polydata_cfg.file_path)
    
        for image_cfg in polydata_cfg.images:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                take_snapshot(
                    scalar_name=image_cfg.scalar_label,
                    file_path=vtp_path,
                    output_path=output_path / f"{image_cfg.image_label}.png",
                    colormap_params=cfg.colormap,
                    projection_params=cfg.projection,
                    camera_params=cfg.camera,
                )
            process_image(
                image_path=output_path / f"{image_cfg.image_label}.png",
                output_path=output_path / f"{image_cfg.image_label}.png",
                crop_cfg=cfg.camera.crop,
            )

In [3]:
import PIL
from PIL import Image
import pathlib

def paste_watermark(main_image: Image, watermark_image: Image):
    """Adds a watermark to the main image

    Args:
        main_image (Image): Main Image
        watermark_image (Image): Watermark image
    """
    OFFSET = 40
    position = (main_image.size[0] - watermark_image.size[0] - OFFSET, main_image.size[1] - watermark_image.size[1] - OFFSET)
    main_image.paste(
        watermark_image,
        position,
        watermark_image,
    )

images_path = pathlib.Path("/home/ubuntu/Desktop/slice_animation")
watermark_path = pathlib.Path("/home/ubuntu/Documentos/Repositories/insightv/insight/pressure/static/img/white_logo.png")

watermark_img = Image.open(watermark_path)
w_size = (s * 3 for s in watermark_img.size)
watermark_img = watermark_img.resize(w_size)

image_prefix = "slice_animation"

files_list = [f for f in images_path.iterdir() if f.is_file()]
files_list = [f for f in files_list if f.name.startswith(image_prefix)]

for file in files_list:
    t_img = Image.open(file)
    paste_watermark(t_img, watermark_img)
    t_img.save(file.parent / ("watermarked_" + file.name))
    print(file, file.parent / ("watermarked_" + file.name))

/home/ubuntu/Desktop/slice_animation/slice_animation.0025.png /home/ubuntu/Desktop/slice_animation/watermarked_slice_animation.0025.png
/home/ubuntu/Desktop/slice_animation/slice_animation.0055.png /home/ubuntu/Desktop/slice_animation/watermarked_slice_animation.0055.png
/home/ubuntu/Desktop/slice_animation/slice_animation.0058.png /home/ubuntu/Desktop/slice_animation/watermarked_slice_animation.0058.png
/home/ubuntu/Desktop/slice_animation/slice_animation.0081.png /home/ubuntu/Desktop/slice_animation/watermarked_slice_animation.0081.png
/home/ubuntu/Desktop/slice_animation/slice_animation.0151.png /home/ubuntu/Desktop/slice_animation/watermarked_slice_animation.0151.png
/home/ubuntu/Desktop/slice_animation/slice_animation.0096.png /home/ubuntu/Desktop/slice_animation/watermarked_slice_animation.0096.png
/home/ubuntu/Desktop/slice_animation/slice_animation.0194.png /home/ubuntu/Desktop/slice_animation/watermarked_slice_animation.0194.png
/home/ubuntu/Desktop/slice_animation/slice_anima